# Image Filters and Basic Feature Detection
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Edge enhancement](#edge)
* [Spot detection](#spot)
* [Fourier domain](#fourier)

## Introduction
<a id="intro"></a> 

## Edge enhancement
<a id="edge"></a> 

## Spot detection
<a id="spot"></a> 

## Fourier domain
<a id="fourier"></a> 